https://judge.mipt.ru/mipt_cs_on_python3/labs/lab13.html

### Z-функция
Материал позаимствован с сайта e-maxx.ru.\

Определение \
Здесь и далее строки индексируются с нуля, т.е. первый символ строки имеет номер 0
. Также, здесь и далее s[i…j] обозначает подстроку строки s
 от i-го символа до j-го включительно.

Пусть дана строка s длины n. Тогда Z(s)
 - это массив длины n, i-ый элемент которого равен наибольшему числу символов, начиная с позиции i, совпадающих с первыми символами строки s. 

Иными словами, z[i]
 — это длина наибольшего общего префикса (строки s и её i-го суффикса). \
 iй суффикс обязательно начинается с iго символа и не обязан заканчиваться в конце строки

Первый элемент Z
-функции, z[0]
, обычно считают неопределённым. В данной статье мы будем считать, что он равен нулю (хотя ни в алгоритме, ни в приведённой реализации это ничего не меняет).

Далее будет привиден алгоритм вычисления Z
-функции за время O(n)
, а также различные применения этого алгоритма.

##### Примеры
Приведём для примера подсчитанную Z-функцию для нескольких строк: \
"aaaaa": \
z[0] = 0, \
z[1] = 4, \
z[2] = 3, \
z[3] = 2, \
z[4] = 1. \
"aaabaab": \
z[0] = 0, по определению \
z[1] = 2, "aaabaab", 1a2a совпадает с 0а1а \
z[2] = 1, 2a совпадает с 0а \
z[3] = 0, 3b не совпадает с 0a \
z[4] = 2, 4a5a == 0a1a\
z[5] = 1, 5a==0a\
z[6] = 0. 6b!=0a\
"abacaba": \
z[0] = 0, \
z[1] = 0, \
z[2] = 1, \
z[3] = 0, \
z[4] = 3, \
z[5] = 0, \
z[6] = 1. 


##### Тривиальный алгоритм
Формальное определение можно представить в виде следующей элементарной реализации за O(n2)
Мы просто для каждой позиции i
 перебираем ответ для неё z[i]
, начиная с нуля, и до тех пор, пока мы не обнаружим несовпадение или не дойдём до конца строки.

Разумеется, эта реализация слишком неэффективна, перейдём теперь к построению эффективного алгоритма.




In [ ]:
def z_func_triv(s:str, n):
    z = [0] * n
    for i in range(1, n):
        while i + z[i] < n and s[z[i]] == s[i + z[i]]:
            z[i] += 1
    return z
print(z_func_triv("abacaba",7))
z_func_triv("abcabc",6)

#### Эффективный алгоритм вычисления Z-функции O(N)
Чтобы получить эффективный алгоритм, будем вычислять значения z[i] по очереди — от i=1 до n−1 \
и при этом постараемся при вычислении очередного значения z[i] максимально использовать уже вычисленные значения. \

Назовём для краткости подстроку, совпадающую с префиксом строки s, **отрезком совпадения**. Например, значение искомой Z-функции z[i] — это длина длиннейшего отрезок совпадения, начинающийся в позиции i (и заканчиваться он будет в позиции i+z[i]−1).

Для этого будем поддерживать **координаты [l;r] самого правого отрезка совпадения**, т.е. из всех обнаруженных отрезков будем хранить тот, который оканчивается правее всего. В некотором смысле, индекс r — это такая граница, до которой наша строка уже была просканирована алгоритмом, а всё остальное — пока ещё не известно.

Тогда если текущий индекс, для которого мы хотим посчитать очередное значение Z-функции, — это i, мы имеем один из двух вариантов: 
1) i>r — т.е. текущая позиция лежит **за пределами** того, что мы уже успели обработать.

Тогда будем искать z[i] **тривиальным алгоритмом**, т.е. просто пробуя значения z[i]=0, z[i]=1
, и т.д. \
Заметим, что в итоге, если z[i] окажется >0, то мы будем обязаны **обновить координаты самого правого отрезка [l;r]** — т.к. i+z[i]−1 гарантированно окажется больше r 

2) i≤r
 — т.е. текущая позиция лежит **внутри** отрезка совпадения [l;r]
.
Тогда мы можем использовать уже подсчитанные **предыдущие** значения Z-функции, чтобы проинициализировать значение z[i] не нулём, а каким-то возможно бОльшим числом.

Для этого заметим, что подстроки s[l…r] и s[0…r−l] **совпадают**. Это означает, что в качестве **начального приближения** для z[i] можно взять соответствующее ему значение из отрезка s[0…r−l], а именно, значение z[i−l].

Однако значение z[i−l] могло оказаться **слишком большим**: таким, что при применении его к позиции i оно "вылезет" за пределы границы r. Этого допустить нельзя, т.к. про символы правее r мы ничего не знаем, и они могут отличаться от требуемых.

###### Приведём пример такой ситуации, на примере строки "aaaabaa".
0a1a2a3a4b5a6a\
i=1 \
1й случай \
lr=[00] \
Итого z[1]=3 \
s[lr]=aaa=s[1,3] \
i=2 \
2й случай \ i<r==3
0а1а2а=s[0…r−l]=1а2а3а=s[lr] \
z[i−l]=z[1]=3 \
i+3>r-не подходит \
$ z0[i]=min(r−i+1,z[i−l])$
z0[i]=2
короче кусочек 2а3а совпадает с 1а2а  \
z[2]=2 \
Когда мы дойдём до последней позиции (i=6), текущим самым правым отрезком будет [l;r]=[5;6]. \ Позиции 6 с учётом этого отрезка будет соответствовать позиция 6−5=1, ответ в которой равен z[1]=3. Очевидно, что таким значением инициализировать z[6] нельзя, оно совершенно некорректно. Максимум, каким значением мы могли проинициализировать — это 1
, поскольку это наибольшее значение, которое не вылезает за пределы отрезка [l;r]
Таким образом, в качестве начального приближения для z[i] безопасно брать только такое выражение:

$ z0[i]=min(r−i+1,z[i−l])$. 

Проинициализировав z[i] таким значением z0[i], мы снова дальше действуем тривиальным алгоритмом — потому что после границы r, вообще говоря, могло обнаружиться продолжение отрезка совпадение, предугадать которое одними лишь предыдущими значениями Z-функции мы не можем.

Таким образом, весь алгоритм представляет из себя два случая, которые фактически различаются только начальным значением z[i]
: в первом случае оно полагается равным нулю, а во втором — определяется по предыдущим значениям по указанной формуле. После этого обе ветки алгоритма сводятся к выполнению тривиального алгоритма, стартующего сразу с указанного начального значения.

Алгоритм получился весьма простым. Несмотря на то, что при каждом i
 в нём так или иначе выполняется тривиальный алгоритм — мы достигли существенного прогресса, получив алгоритм, работающий за линейное время (действительно, на каждый символ мы "посмотрим", т.е. сравним его с каким-либо предыдущим всего один раз).

Упражнение №1: Z-функция
Напишите Z-функцию. Пусть заголовком ее будет def z_func(s, n):

In [1]:
def z_func(s:str, n:int):
    """s-строка,n-длина строки"""
    l=r=0
    z=[0]*n
    for i in range(1,n):
        if i<=r:
            z[i]= min(r-i+1,z[i-l])#инициализируем начальное приближение
        while i+z[i]<n and s[z[i]]==s[i+z[i]]:
            z[i]+=1
        if z[i]>0:
            l=i
            r=i+z[i]-1
    return z

def test_z_func(z_func):
    strings=["aaaaa","aaabaab","abacaba","aaaabaa","abcafabcabcaf"]
    answers=[[0,4,3,2,1],[0,2,1,0,2,1,0],[0,0,1,0,3,0,1],[0,3,2,1,0,2,1],[0,0,0,1,0,4,0,0,5,0,0,1,0]]
    for i in range(len(strings)):
        res=z_func(strings[i],len(strings[i]))
        print("Ok" if res==answers[i] else "Fail",strings[i],res)
test_z_func(z_func)
z_func("abcabc",6)

Ok aaaaa [0, 4, 3, 2, 1]
Ok aaabaab [0, 2, 1, 0, 2, 1, 0]
Ok abacaba [0, 0, 1, 0, 3, 0, 1]
Ok aaaabaa [0, 3, 2, 1, 0, 2, 1]
Ok abcafabcabcaf [0, 0, 0, 1, 0, 4, 0, 0, 5, 0, 0, 1, 0]


[0, 0, 0, 3, 0, 0]

Упражнение №2: Поиск подстроки
Пусть даны две строки. Найти все вхождения второй строки в первую.

In [2]:
import string
def search_substr(s,sub_s):
    """Search substring in string s, returns list of entering indexes"""
    search_s=sub_s+s
    n=len(search_s)
    z=z_func(search_s,n)
    index=[]
    sub_s_len=len(sub_s)
    for i in range(sub_s_len,n):
        if sub_s_len<=z[i]:
            index.append(i-sub_s_len)
    return index

def get_connected_symbol(s:str):
    """Finds a symbol wich isn't in string"""
    charlist=['#','$','*','&','^']
    for char in charlist:
        if not char in s:
            return char
    for char in string.printable: # string.printable:
        if not char in s:
            return char
    return -1

def search_substr_symbol(s,sub_s): #O(N)
    """Search substring in string s, returns list of entering indexes,
    uses a symbol wich isn't in string"""
    char=get_connected_symbol(sub_s+s)
    search_s=sub_s+char+s
    print(search_s)
    n=len(search_s)
    z=z_func(search_s,n)
    index=[]
    sub_s_len=len(sub_s)+len(char)
    for i in range(sub_s_len,n):
        if z[i]==len(sub_s):
            index.append(i-sub_s_len)
    return index

def test_search_substr(search_substr):
    sub_s=["aba","bab","a#$^&*","a#$^*"]
    s=["cabafabaab","babbabfbababf","ba#$^&*","ba#$^*"]
    answer=[[1,5],[0,3,7,9],[1],[1]]
    for i in range(len(sub_s)):
        res=search_substr(s[i],sub_s[i])
        print("Ok" if res==answer[i] else "Fail",sub_s[i],s[i],res)

In [3]:
test_search_substr(search_substr)
test_search_substr(search_substr_symbol)

Ok aba cabafabaab [1, 5]
Ok bab babbabfbababf [0, 3, 7, 9]
Ok a#$^&* ba#$^&* [1]
Ok a#$^* ba#$^* [1]
aba#cabafabaab
Ok aba cabafabaab [1, 5]
bab#babbabfbababf
Ok bab babbabfbababf [0, 3, 7, 9]
a#$^&*0ba#$^&*
Ok a#$^&* ba#$^&* [1]
a#$^*&ba#$^*
Ok a#$^* ba#$^* [1]


Упражнение №3: Количество разных подстрок
Найти число всех различных подстрок входящих в данную.

Количество различных подстрок в строке
Дана строка s длины n. Требуется посчитать количество её различных подстрок.

Будем решать эту задачу итеративно. А именно, научимся, зная текущее количество различных подстрок, пересчитывать это количество при добавлении в конец одного символа.

Итак, пусть k — текущее количество различных подстрок строки s, и мы добавляем в конец символ c. Очевидно, в результате могли появиться некоторые новые подстроки, оканчивавшиеся на этом новом символе c (а именно, все подстроки, оканчивающиеся на этом символе, но не встречавшиеся раньше).

Возьмём строку t=s+c и инвертируем её (запишем символы в обратном порядке). Наша задача — посчитать, сколько у строки t таких префиксов, которые не встречаются в ней более нигде. Но если мы посчитаем для строки t Z-функцию и найдём её максимальное значение $z_{\rm max}$, то, очевидно, в строке t встречается (не в начале) её префикс длины $z_{\rm max}$, но не большей длины. Понятно, префиксы меньшей длины уже точно встречаются в ней.

Итак, мы получили, что число новых подстрок, появляющихся при дописывании символа c, равно $len - z_{\rm max}$, где len — текущая длина строки после приписывания символа c.

Следовательно, асимптотика решения для строки длины n составляет O (n^2).

Стоит заметить, что совершенно аналогично можно пересчитывать за O (n) количество различных подстрок и при дописывании символа в начало, а также при удалении символа с конца или с начала.

In [4]:
def count_substrings(s:str):
    """Counts number of different substrings in string"""
    n=len(s)
    not_count=[0]*n
    i=0
    while len(s)>1:
        z=z_func(s,len(s))
        for j in range(i,n):
            not_count[j]=max(z[j-i],not_count[j])
        s=s[1:]#remove first char
        i+=1
    count=sum(range(2,n+1))-sum(not_count)#if subs is string from 1
    return count

In [5]:
def count_substrings_2(s:str):
    """Принципиально не отличается, будем дописывать символы в конец строки
    и идти по возрастанию строки,не нужно искать max префикс среди всех,
    для каждого дописанного символа отдельно"""
    n=len(s)
    i=0
    count=0 #для строки с единственным символом
    for i in range(1,n):
        s_new=s[i::-1]
        z=z_func(s_new,len(s_new))
        count+=len(s_new)-max(z)
    return count
        

In [6]:
def test_count_substr(count_substrings):
    s=["aaaaa","aaabaab","abacaba","aaaabaa","abcafabcabcaf"]
    answer=[4,18,20,18,65]
    for i in range(len(s)):
        res=count_substrings(s[i])
        print("Ok" if res==answer[i] else "Fail",s[i],res)

In [7]:
test_count_substr(count_substrings_2)

Ok aaaaa 4
Ok aaabaab 18
Ok abacaba 20
Ok aaaabaa 18
Ok abcafabcabcaf 65


Упражнение №4: Период строки
Для данной строки s найти строку p минимальной длины, такую что s можно предстваить как конкатенацию одной или нескольких копий p. \
s="abababab" p="ab" \
Пусть s=p+...+p ,k раз \
Тогда len(s)=k*len(p) \
Тогда $z(s)=[0,...(k-1)*len(p)....(k-2)*len(p)...len(p)...]$ \
для индексов $i=[0..len(p)+1...2*len(p)+1...(k-1)*len(p) ]$ \
причем  $(k-1)*len(p)=max(z[i]), i=len(p)$ - отсюда находим i и k и проверяем структуру\
p=s[:len(p)+1]

In [8]:
def find_string_period(s:str):
    """Finds min period of string p: p*k=s, k=1,2..."""
    n=len(s)
    z=z_func(s,n)
    max_z=0
    len_p=0
    #find max z
    for i in range(n):
        if z[i]>max_z:
            max_z=z[i]
            len_p=i
    # find k
    if max_z+len_p==n:#максимальный совпадающий префикс идет до конца строки
        return len_p
    return n

In [9]:
def test_find_period(find_period):
    s=["ababab","aaaaa","abcabc","abc","","abababcd"]
    answer=[2,1,3,3,0,8]
    for i in range(len(s)):
        res=find_period(s[i])
        print("Ok" if res==answer[i] else "Fail",res,s[i])

In [10]:
test_find_period(find_string_period)

Ok 2 ababab
Ok 1 aaaaa
Ok 3 abcabc
Ok 3 abc
Ok 0 
Ok 8 abababcd


##### Префикс-функция. Определение
Пусть дана строка s длины n. Тогда π(s) - это массив длины n, i-ый элемент которого (π[i]
) определяется следующим образом: \
это длина наибольшего собственного суффикса подстроки s[0…i], совпадающего с её префиксом (собственный суффикс — значит не совпадающий со всей строкой). В частности, значение π[0] полагается равным нулю. \
Суффикс обязательно заканчивается на iм элементе \

Примечение: вообще говоря, в теории множеств собственным считается не пустое подмножество, не совпдающее с самим множеством. В данной статье, для простоты суффикс и префикс нулевой длины также считаются собственными.

Математически определение префикс-функции можно записать следующим образом:

$π[i]=maxk=0…i{k:s[0…k−1]=s[i−k+1…i]}$. \
Например, для строки "abcabcd" префикс-функция равна: [0,0,0,1,2,3,0]
, что означает: \
у строки "a" нет нетривиального префикса, совпадающего с суффиксом; \
у строки "ab" нет нетривиального префикса, совпадающего с суффиксом; \
у строки "abc" нет нетривиального префикса, совпадающего с суффиксом; \
у строки "abca" префикс длины 1 совпадает с суффиксом; \
у строки "abcab" префикс длины 2 совпадает с суффиксом; \
у строки "abcabc" префикс длины 3 совпадает с суффиксом; \
у строки "abcabcd" нет нетривиального префикса, совпадающего с суффиксом. \
Другой пример — для строки "aabaaab" она равна: [0,1,0,1,2,2,3] \


##### Тривиальный алгоритм
Непосредственно следуя определению, можно написать такой алгоритм вычисления префикс-функции:
Как нетрудно заметить, работать он будет за O(n3) , что слишком медленно.

In [11]:
def prefix_func_triv(s, n):
    pi = [0] * n
    for i in range(n - 1):
        for k in range(1, i + 1):
            equal = True
            for j in range(k):
                if s[j] != s[i - k  + 1 + j]:
                    equal = False
                    break
            if equal:
                pi[i] = k
    return pi

#### Эффективный алгоритм Пи-функции (см лекции)
$\pi=$[0 для всех i], \
для всех i строки s: \
$ \;\;\;\;  p=\pi_{s_{i-1}}$  #пи-функции строки без последнего символа \
$ \;\;\;\;$ пока p>0 и $s_i!=s_{p+1}$ # сравниваем последний эл-т строки si с эл-том после префикса \
$ \;\;\;\;\;\;\;\; p=\pi_{s_p} $  # берем меньший префик(префик префикса) \
$ \;\;\;\;$ Если $ s_i==s_{p+1}$,то элемент совпал\
$ \;\;\;\;\;\;\;\; p+=1 $ \
$ \;\;\;\; \pi_i=p $ 

Упражнение №5: Префикс-функция
Напишите префикс-функцию. Пусть заголовком ее будет def p_func(s, n):

In [12]:
def p_func(s,n):
    Pi=[0 for i in range(n)]
    for i in range(1,n):
        p=Pi[i-1]
        while p>0 and s[i]!=s[p]:
            p=Pi[p-1]
        if s[i]==s[p]:
            p+=1
        Pi[i]=p
    return Pi

In [13]:
def test_p_func(p_func):
    strings=["aaaaa","aaabaab","abacaba","aabaaab"]
    answers=[[0,1,2,3,4],[0,1,2,0,1,2,0],[0,0,1,0,1,2,3], [0,1,0,1,2,2,3]]
    for i in range(len(strings)):
        res=p_func(strings[i],len(strings[i]))
        print("Ok" if res==answers[i] else "Fail",strings[i],res)
test_p_func(p_func)

Ok aaaaa [0, 1, 2, 3, 4]
Ok aaabaab [0, 1, 2, 0, 1, 2, 0]
Ok abacaba [0, 0, 1, 0, 1, 2, 3]
Ok aabaaab [0, 1, 0, 1, 2, 2, 3]


Упражнение №6: Поиск подстроки
Пусть даны две строки. Найти все вхождения второй строки в первую с помощью алгоритма Кнута-Морриса-Пратта.


Эта задача является классическим применением префикс-функции (и, собственно, она и была открыта в связи с этим).

Дан текст t
 и строка s
, требуется найти и вывести позиции всех вхождений строки s
 в текст t
.

Обозначим для удобства через n
 длину строки s
, а через m
 — длину текста t
.

Образуем строку s+#+t
, где символ #
 — это разделитель, который не должен нигде более встречаться. Посчитаем для этой строки префикс-функцию. Теперь рассмотрим её значения, кроме первых n+1
 (которые, как видно, относятся к строке s
 и разделителю). По определению, значение π[i]
 показывает наидлиннейшую длину подстроки, оканчивающейся в позиции i
 и совпадающего с префиксом. Но в нашем случае это π[i]
 — фактически длина наибольшего блока совпадения со строкой s
 и оканчивающегося в позиции i
. Больше, чем n
, эта длина быть не может, за счёт разделителя. А вот равенство π[i]=n
 (там, где оно достигается), означает, что в позиции i
 оканчивается искомое вхождение строки s
 (только не надо забывать, что все позиции отсчитываются в склеенной строке s+#+t
).

Таким образом, если в какой-то позиции i
 оказалось π[i]=n
, то в позиции i−(n+1)−n+1=i−2n
 строки t
 начинается очередное вхождение строки s
 в строку t
.

Как уже упоминалось при описании алгоритма вычисления префикс-функции, если известно, что значения префикс-функции не будут превышать некоторой величины, то достаточно хранить не всю строку и префикс-функцию, а только её начало. В нашем случае это означает, что нужно хранить в памяти лишь строку s+# и значение префикс-функции (Pi) на ней, а потом уже считывать по одному символу строку t и пересчитывать текущее значение префикс-функции(p) \
Изначально почитанное Pi не меняем!!!

Итак, алгоритм Кнута-Морриса-Пратта решает эту задачу за O(n+m)
 времени и O(n)
 памяти.

In [14]:
def seach_substr_KMP_easy(s,sub_s,sep="#"):
    """ врямя O(N+M) память O(N+M)"""
    s=sub_s+sep+s
    Pi=p_func(s,len(s))
    index=[]
    for i in range(len(sub_s)+len(sep),len(Pi)):
        if Pi[i]==len(sub_s):
            index.append(i-2*len(sub_s)-len(sep)+1)
    return index

In [15]:
def seach_substr_KMP(s,sub_s,sep="#"):
    """Classic KMP time - O(N+M), memory O(M)"""
    sub_s_sep=sub_s+sep
    m=len(sub_s_sep)
    n=len(s)
    index=[]
    Pi=p_func(sub_s_sep,m)
    p=0
    for i in range(n):
        #print(p)
        while p>0 and s[i]!=sub_s_sep[p]:
            p=Pi[p-1]
        if s[i]==sub_s[p]:
            p+=1
        if p==len(sub_s):
            index.append(i-len(sub_s)+1)
    return index
    
    

In [16]:
test_search_substr(seach_substr_KMP_easy)
test_search_substr(seach_substr_KMP)
seach_substr_KMP("baaaaa","aaa")

Ok aba cabafabaab [1, 5]
Ok bab babbabfbababf [0, 3, 7, 9]
Ok a#$^&* ba#$^&* [1]
Ok a#$^* ba#$^* [1]
Ok aba cabafabaab [1, 5]
Ok bab babbabfbababf [0, 3, 7, 9]
Ok a#$^&* ba#$^&* [1]
Ok a#$^* ba#$^* [1]


[1, 2, 3]

Упражнение №7: Поиск подстроки онлайн
В первой строке ввода - число n
, количество букв в паттерне. Во второй строке - паттерн, строка которую нужно искать в тексте. В каждой из последующих строк - символы текста, по одному в каждой строке. Необходимо вывести позиции вхождений паттерна в текст. Длина текста заранее не известна, он может быть очень большим.
Для простоты будем считывать из файла

In [17]:
def search_substr_online(sub_s,n,file_name):
    file = open(file_name, 'r')
    if not file or sub_s=="":
        return []
    Pi=p_func(sub_s,len(sub_s))
    i=p=0
    index=[]
    while True: 
    # read by character
        char = file.read(1)
        if not char:
            file.close()
            break
        while p>0 and char!=sub_s[p]:
            p=Pi[p-1]
        if char==sub_s[p]:
            p+=1
        if p==len(sub_s):
            p=Pi[p-1]
            index.append(i-len(sub_s)+1)
        i+=1
    return index

In [18]:
from pathlib import Path
path_to_data=Path("../data")
def test_search_subs_online():
    path=path_to_data.joinpath("test_KMP.txt")
    f=open(path,"r")
    print(f.read())
    f.close()
    sub_s=["Hello","o","world","Python",""]
    for s in sub_s:
        res=search_substr_online(s,len(s),path)
        print(s,res)

In [19]:
test_search_subs_online()

Hello, world!
Hello, Python!
It's Python world!

Hello [0, 14]
o [4, 8, 18, 25, 38, 42]
world [7, 41]
Python [21, 34]
 []


Упражнение №8: Количество разных подстрок
Найти число всех различных подстрок входящих в данную с помощью префикс-функции.

In [20]:
def count_substr_pi_func(s):
    """Counst number of different substrings in string using Pi-function"""
    n=len(s)
    count=0
    for i in range(1,n):
        sub_s=s[i::-1]
        count+=len(sub_s)-max(p_func(sub_s,len(sub_s)))
    return count

In [21]:
test_count_substr(count_substr_pi_func)

Ok aaaaa 4
Ok aaabaab 18
Ok abacaba 20
Ok aaaabaa 18
Ok abcafabcabcaf 65


Упражнение №9: Период строки
Для данной строки s найти строку p минимальной длины, такую что s можно предстваить как конкатенацию одной или нескольких копий p
. Используйте префикс-функцию.

In [22]:
def find_string_period_pi(s:str):
    """Finds min period of string p: p*k=s, k=1,2...using Pi-function"""
    if s=="":
        return s
    n=len(s)
    #if s=p*k pk_1=(k-1)p
    p_max=p_func(s,n)[n-1]
    len_p=n-p_max
    if p_max%len_p==0:
        return s[:len_p]
    return s

In [23]:
def test_find_period_2(find_period):
    s=["ababab","aaaaa","abcabc","abc","","abababcd"]
    #answer=[2,1,3,3,0,8]
    answer=["ab","a","abc","abc","","abababcd"]
    for i in range(len(s)):
        res=find_period(s[i])
        print("Ok" if res==answer[i] else "Fail",res,s[i])

In [24]:
test_find_period_2(find_string_period_pi)

Ok ab ababab
Ok a aaaaa
Ok abc abcabc
Ok abc abc
Ok  
Ok abababcd abababcd
